# Importing and Dropping Duplicates

In [29]:
import pandas as pd
import json

In [30]:
imports = []
for i in range(1,41):
    imports.append('genre_exports/export_%s.json' %(str(i)))   #rename to genre_export

In [31]:
imports

['genre_exports/export_1.json',
 'genre_exports/export_2.json',
 'genre_exports/export_3.json',
 'genre_exports/export_4.json',
 'genre_exports/export_5.json',
 'genre_exports/export_6.json',
 'genre_exports/export_7.json',
 'genre_exports/export_8.json',
 'genre_exports/export_9.json',
 'genre_exports/export_10.json',
 'genre_exports/export_11.json',
 'genre_exports/export_12.json',
 'genre_exports/export_13.json',
 'genre_exports/export_14.json',
 'genre_exports/export_15.json',
 'genre_exports/export_16.json',
 'genre_exports/export_17.json',
 'genre_exports/export_18.json',
 'genre_exports/export_19.json',
 'genre_exports/export_20.json',
 'genre_exports/export_21.json',
 'genre_exports/export_22.json',
 'genre_exports/export_23.json',
 'genre_exports/export_24.json',
 'genre_exports/export_25.json',
 'genre_exports/export_26.json',
 'genre_exports/export_27.json',
 'genre_exports/export_28.json',
 'genre_exports/export_29.json',
 'genre_exports/export_30.json',
 'genre_exports/exp

In [32]:
movies_df = pd.DataFrame()

In [33]:
for imp in imports:
    movies_df = movies_df.append(pd.read_json(imp, orient='index'))

# mov = mov.append(pd.read_json('export_2.json', orient='index'))
# mov = mov.append(pd.read_json('export_3.json', orient='index'))

In [34]:
len(movies_df)

188326

### Use New DataFrame

In [35]:
movies = pd.DataFrame(movies_df)

In [36]:
movies['ID'] = movies.index

In [37]:
movies = movies.drop_duplicates(subset='ID',keep='first')

In [38]:
len(movies)

106906

## Drop NaN Ratings

In [39]:
movies['Rating'].isnull().sum()

29609

In [40]:
movies = movies.dropna(subset = ['Rating'])

In [41]:
len(movies)

77297

## Keep Only Movies with Drop Movies with 1000 Votes or More

In [42]:
movies['Votes'] = movies['Votes'].str.replace(',','')

In [43]:
movies['Votes']

http://www.imdb.com/title/tt0006206/     3519
http://www.imdb.com/title/tt0009682/      541
http://www.imdb.com/title/tt0014720/        9
http://www.imdb.com/title/tt0015163/     7287
http://www.imdb.com/title/tt0015324/    24694
http://www.imdb.com/title/tt0017925/    61402
http://www.imdb.com/title/tt0018578/     8998
http://www.imdb.com/title/tt0019421/    10630
http://www.imdb.com/title/tt0019532/      269
http://www.imdb.com/title/tt0020815/     1325
http://www.imdb.com/title/tt0021079/     9804
http://www.imdb.com/title/tt0022753/      728
http://www.imdb.com/title/tt0023427/    21444
http://www.imdb.com/title/tt0023551/     5944
http://www.imdb.com/title/tt0025004/     1958
http://www.imdb.com/title/tt0025862/     4124
http://www.imdb.com/title/tt0026164/     1433
http://www.imdb.com/title/tt0026174/    10768
http://www.imdb.com/title/tt0027438/     3741
http://www.imdb.com/title/tt0027623/     1745
http://www.imdb.com/title/tt0029047/     1707
http://www.imdb.com/title/tt002908

In [44]:
movies['Votes'] = movies['Votes'].astype(int)

In [45]:
len(movies[movies['Votes'] >= 500])

29717

In [46]:
movies = movies[movies['Votes'] >= 500]

# Scrape Details with Movie URL

In [47]:
len(movies.index)

29717

In [48]:
movies.index

Index(['http://www.imdb.com/title/tt0006206/',
       'http://www.imdb.com/title/tt0009682/',
       'http://www.imdb.com/title/tt0015163/',
       'http://www.imdb.com/title/tt0015324/',
       'http://www.imdb.com/title/tt0017925/',
       'http://www.imdb.com/title/tt0018578/',
       'http://www.imdb.com/title/tt0019421/',
       'http://www.imdb.com/title/tt0020815/',
       'http://www.imdb.com/title/tt0021079/',
       'http://www.imdb.com/title/tt0022753/',
       ...
       'http://www.imdb.com/title/tt0439504/',
       'http://www.imdb.com/title/tt0457530/',
       'http://www.imdb.com/title/tt0495747/',
       'http://www.imdb.com/title/tt1384925/',
       'http://www.imdb.com/title/tt1388402/',
       'http://www.imdb.com/title/tt1555110/',
       'http://www.imdb.com/title/tt2014202/',
       'http://www.imdb.com/title/tt2140381/',
       'http://www.imdb.com/title/tt4893452/',
       'http://www.imdb.com/title/tt5143890/'],
      dtype='object', length=29717)

In [49]:
movie_urls = list(movies.index)

In [50]:
len(movie_urls)

29717

In [51]:
movie_urls = sorted(movie_urls,reverse = True)

In [52]:
movies.columns

Index(['Certificate', 'Director', 'Genre', 'Gross', 'Metascore', 'Rating',
       'Runtime', 'Star1', 'Star1_URL', 'Star2', 'Star2_URL', 'Star3',
       'Star3_URL', 'Star4', 'Star4_URL', 'Title', 'Votes', 'Year', 'ID'],
      dtype='object')

In [54]:
# Movie Details
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

movies_scraped = 0
movie_details = {}
movie_index = 27000                       
prefix = 'http://www.imdb.com'
export_number = 45
export_no = str(export_number)
load_attempts = 0

def movie_scraper():
    global movie_index
    
    print(str(datetime.now()),': working on export ' + export_no)
    
    for url in movie_urls[movie_index:]:
        if (movie_index+1) % 20 == 0:
            time.sleep(1.5)
            print('%d: %s' %(movie_index+1, url))
        page = request_page(url)
        get_movie_details(url, page)
        movie_index += 1
    
    export_to_json(movie_details)
    print("Done")
    
def request_page(url):
    global load_attempts
    response = requests.get(url)
    
    if response.status_code == 200:
        return(response.text)
        load_attempts = 0
    else:
        print(response.status_code,'for %s' %(url))
        if response.status_code != 404:
            if load_attempts == 10:
                time.sleep(60)
            else:
                time.sleep(10)
            request_page(url)
            load_attempts += 1

def get_movie_details(url, page):
    soup = BeautifulSoup(page, 'html.parser')

    movie_details[url] = {}
    
    plot_wrapper = soup.find('div', class_='plot_summary_wrapper')

    if plot_wrapper:
        if plot_wrapper.find(text=re.compile('Director:')):
            directorURL = plot_wrapper.find(text=re.compile('Director:'))
        elif plot_wrapper.find(text=re.compile('Directors:')):
            directorURL = plot_wrapper.find(text=re.compile('Directors:'))
        else:
            directorURL = None
    directorURL = directorURL.parent.parent if directorURL else None
    directorURL = directorURL.find('a') if directorURL else None
    director = directorURL.text if directorURL else None
    directorURL = directorURL['href'] if directorURL else None
    directorURL = prefix+directorURL.split("?")[0] if directorURL else None
    movie_details[url]['Director'] = director
    movie_details[url]['Director_URL'] = directorURL

    if plot_wrapper:
        if plot_wrapper.find(text=re.compile('Stars:')):
            starURLs = plot_wrapper.find(text=re.compile('Stars:'))
        elif plot_wrapper.find(text=re.compile('Star:')):
            starURLs = plot_wrapper.find(text=re.compile('Star:'))
        else:
            starURLs = None
    starURLs = starURLs.parent.parent if starURLs else None
    starURLs = starURLs.findAll('a') if starURLs else None
    if starURLs:
        if len(starURLs) == 4:
            starURLs = [[star['href'],star.text] for star in starURLs[:-1]]
        else:
            starURLs = [[star['href'],star.text] for star in starURLs]
    if starURLs:
        i = 0
        while len(starURLs) >= i+1:
            for URL in starURLs:
                star = starURLs[i]
                movie_details[url]['Star%s' %str(i+1)] = star[1].split('\n')[0].strip() if star[1] else None
                movie_details[url]['Star%s_URL' %str(i+1)] = prefix+star[0].split("?")[0] if star[0] else None
            i += 1 
    
    title = soup.find('div', class_='title_wrapper').find('h1',itemprop='name')
    title = title.contents[0].replace(u'\xa0', u'')
    movie_details[url]['Title'] = title

    release_date = soup.find('div', class_='title_wrapper')
    release_date = release_date.findAll('a') if release_date else None
    release_date = release_date[-1] if release_date else None
    release_date = release_date.find('meta') if release_date else None
    release_date = str(release_date) if release_date else None
    release_date = re.sub('[^0-9-]+','', release_date) if release_date else None
    movie_details[url]['Release Date'] = release_date

    title_details = soup.find(id='titleDetails')
    
    production_company = title_details.find(text=re.compile('Production Co:')) if title_details else None
    production_company = production_company.parent.parent if production_company else None
    production_company = production_company.find('a') if production_company else None
    production_company = production_company.text if production_company else None
    movie_details[url]['Production Company'] = production_company

    country = title_details.find(text=re.compile('Country:')) if title_details else None
    country = country.parent.parent if country else None
    country = country.find('a') if country else None
    country = country.text if country else None
    movie_details[url]['Country'] = country

    language = title_details.find(text=re.compile('Language:')) if title_details else None
    language = language.parent.parent if language else None
    language = language.find('a') if language else None
    language = language.text if language else None
    movie_details[url]['Language'] = language

    primary_filming_location = title_details.find(text=re.compile('Filming Locations:')) if title_details else None
    primary_filming_location = primary_filming_location.parent.parent if primary_filming_location else None
    primary_filming_location = primary_filming_location.find('a') if primary_filming_location else None
    primary_filming_location= primary_filming_location.text if primary_filming_location else None
    movie_details[url]['Primary Filming Location'] = primary_filming_location

    budget = title_details.find(text=re.compile('Budget:')) if title_details else None
    budget = budget.parent.parent if budget else None
    budget = budget.text.strip() if budget else None
    budget = re.sub('[^0-9]+','', budget) if budget else None
    movie_details[url]['Budget'] = budget

    opening_weekend_usa = title_details.find(text=re.compile('Opening Weekend USA:')) if title_details else None
    opening_weekend_usa = opening_weekend_usa.parent.parent if opening_weekend_usa else None
    opening_weekend_usa = opening_weekend_usa.text.strip() if opening_weekend_usa else None
    opening_weekend_usa = re.sub('[^0-9]+','', opening_weekend_usa) if opening_weekend_usa else None
    movie_details[url]['Opening Weekend USA'] = opening_weekend_usa

    #rename gross to gross USA in listing scrape
    global_gross = title_details.find(text=re.compile('Cumulative Worldwide Gross:')) if title_details else None
    global_gross = global_gross.parent.parent if global_gross else None
    global_gross = global_gross.text.strip() if global_gross else None
    global_gross = re.sub('[^0-9]+','', global_gross) if global_gross else None
    movie_details[url]['Cumulative Worldwide Gross'] = global_gross

    gross_usa = title_details.find(text=re.compile('Gross USA:')) if title_details else None
    gross_usa = gross_usa.parent.parent if gross_usa else None
    gross_usa = gross_usa.text.strip() if gross_usa else None
    gross_usa = re.sub('[^0-9]+','', gross_usa) if gross_usa else None
    movie_details[url]['Gross USA'] = gross_usa

    go_to_next_and_export()

def go_to_next_and_export():
    global movies_scraped
    global movie_details
    
    movies_scraped+=1

    if len(movie_details) == 1000:                                         
        export_to_json(movie_details)
        movies_scraped = 0
        movie_details = {}

def export_to_json(movie_details):
    global export_no
    global export_number
    with open('movie_details_export_%s.json' %(export_no),'w')  as f:
        json.dump(movie_details,f)

    export_number += 1
    export_no = str(export_number)
    print(str(datetime.now()),': working on export ' + export_no)

movie_scraper()

2018-01-29 21:04:12.328112 : working on export 45
27020: http://www.imdb.com/title/tt0049263/
27040: http://www.imdb.com/title/tt0049092/
27060: http://www.imdb.com/title/tt0048976/
27080: http://www.imdb.com/title/tt0048764/
27100: http://www.imdb.com/title/tt0048607/
27120: http://www.imdb.com/title/tt0048473/
27140: http://www.imdb.com/title/tt0048316/
27160: http://www.imdb.com/title/tt0048210/
27180: http://www.imdb.com/title/tt0048034/
27200: http://www.imdb.com/title/tt0047889/
27220: http://www.imdb.com/title/tt0047647/
27240: http://www.imdb.com/title/tt0047495/
27260: http://www.imdb.com/title/tt0047376/
27280: http://www.imdb.com/title/tt0047150/
27300: http://www.imdb.com/title/tt0046969/
27320: http://www.imdb.com/title/tt0046828/
27340: http://www.imdb.com/title/tt0046675/
27360: http://www.imdb.com/title/tt0046414/
27380: http://www.imdb.com/title/tt0046247/
27400: http://www.imdb.com/title/tt0046066/
27420: http://www.imdb.com/title/tt0045917/
27440: http://www.imdb.com